## We get imports!

In [102]:
import os
import random
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
import plotly.graph_objects as go
import pandas as pd
from io import BytesIO
from reportlab.platypus import BaseDocTemplate, PageTemplate, Frame
from reportlab.lib.units import inch
from reportlab.platypus import Image, Spacer, Paragraph, KeepTogether
from reportlab.lib.styles import getSampleStyleSheet
from io import BytesIO
import plotly.graph_objects as go
import random
import psycopg2
import pandas as pd
import numpy as np
import ast
import textwrap
import os
import random
import pandas as pd
from reportlab.platypus import Image, Spacer, Paragraph, KeepTogether
from reportlab.lib.styles import getSampleStyleSheet
from io import BytesIO
import plotly.graph_objects as go
import random
from reportlab.platypus import SimpleDocTemplate
from reportlab.lib.styles import ParagraphStyle, getSampleStyleSheet
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont
from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors

## We configure the config

In [103]:
# 1. File Paths
ROBOTO_REGULAR_PATH = '../Report builder/Fonts/Roboto/static/Roboto-Regular.ttf'
ROBOTO_BOLD_PATH = '../Report builder/Fonts/Roboto/static/Roboto-Bold.ttf'
OUTPUT_PDF_PATH = "../Report builder/Reports/report_from_notebook.pdf"
UBUNTU_REGULAR_PATH = '../Report builder/Fonts/Ubuntu/Ubuntu-Regular.ttf'
UBUNTU_BOLD_PATH = '../Report builder/Fonts/Ubuntu/Ubuntu-Bold.ttf'

# Database connection parameters
db_params = {
    "host": "global-vi-backend-production.cpmugytqnb6i.eu-central-1.rds.amazonaws.com",
    "database": "production",
    "user": "powerbi",
    "password": "QKWNRyS3$4X!tK&M"
}


# --- Page Geometry Constants ---
PAGE_WIDTH, PAGE_HEIGHT = letter
MARGIN = 72
USABLE_WIDTH = PAGE_WIDTH - (2 * MARGIN) - 12 # 612 - 144 - 12 = 456

# 2. Charting Colors
COLOR_PALETTE = ['#119d9d', '#48aeb3', '#1a686c', '#8cd7d6', '#4d6063']
HIGHLIGHT_COLOR = '#e86361' 

# 3. Business Logic
selected_company = '40003672631'
selected_company_reg_num = '40003672631'

print(f"Configuration set. True Usable Width is: {USABLE_WIDTH}")

Configuration set. True Usable Width is: 456.0


## We get all the dataframes

In [104]:
#Helper functions for data extranction and cleaning
def execute_sql_query(sql_query: str):
    """
    Connects to a PostgreSQL database, executes a given SQL query,
    and returns the result as a pandas DataFrame.

    Args:
        sql_query (str): The SQL query to be executed.

    Returns:
        pandas.DataFrame: A DataFrame containing the results of the query,
                          or None if an error occurs.
    """
    conn = None
    try:
        # 1. Connect to your PostgreSQL database
        print("Connecting to the PostgreSQL database...")
        conn = psycopg2.connect(**db_params)
        
        # 2. Use pandas to execute the query and load it into a DataFrame
        print(f"Executing query: \"{sql_query}\"")
        df = pd.read_sql_query(sql_query, conn)
        print("Query executed successfully and data loaded into DataFrame.")

        return df

    except (Exception, psycopg2.DatabaseError) as error:
        print(f"Error while connecting to or querying PostgreSQL: {error}")
        return None  # Return None to indicate that the function failed
    finally:
        # 3. Ensure the connection is closed
        if conn is not None:
            conn.close()
            print("Database connection closed.")

In [105]:
#Get jobs df from database
query_to_run = """
WITH "PowerQuerySteps" AS (
    -- Step 1: Source Query from Power Query, selecting initial data
    SELECT
        dj.id,
        dj."createdAt",
        dj."updatedAt",
        dj.name,
        dj."salaryType",
        dj."minSalary",
        dj."maxSalary",
        dj."salaryCurrency",
        dj.deadline,
        dj."organizationName",
        dj."infoUrl",
        dj."latvianRegion", -- Added the original array column as requested
        array_to_string(dj."latvianRegion", ', ') AS "latvianRegionString",

        -- Step 2: Custom Column "IrRīga"
        -- This uses a CASE statement to check if 'Rīga' is an element
        -- in the "latvianRegion" array.
        CASE
            WHEN 'RIGA' = ANY(dj."latvianRegion") THEN 'Rīga'
            ELSE 'Ārpus Rīgas'
        END AS "IrRīga",

        -- Step 3: Custom Column "AverageMonthlySalary"
        CASE
            WHEN dj."salaryType" = 'HOURLY_RATE_BRUTTO' AND dj."minSalary" < 50 AND dj."maxSalary" < 50
            THEN (dj."minSalary" + dj."maxSalary") / 2 * 166
            ELSE (dj."minSalary" + dj."maxSalary") / 2
        END AS "AverageMonthlySalary",

        -- Step 4: Custom Column "Text Between Delimiters"
        REPLACE(SPLIT_PART(dj."infoUrl", '/', 3), 'www.google.com', 'www.cvmarket.lv') AS "Text Between Delimiters"

    FROM
        "DarbnesisJob" dj
    WHERE
        -- Step 5: Initial Filters
        dj."infoUrl" IS NOT NULL AND dj."infoUrl" <> ''
        AND dj.deadline <> '2026-12-31 21:59:59.999'
)
-- Final selection from the CTE
SELECT
    * -- Selects all the columns generated in the CTE above
FROM
    "PowerQuerySteps"
WHERE
    -- Step 6: Final Filter on the calculated salary
    "AverageMonthlySalary" <= 15000
ORDER BY
    -- Step 7: Sorting
    "deadline" DESC;

"""
df_darbnesis_job = execute_sql_query(query_to_run)


Connecting to the PostgreSQL database...
Executing query: "
WITH "PowerQuerySteps" AS (
    -- Step 1: Source Query from Power Query, selecting initial data
    SELECT
        dj.id,
        dj."createdAt",
        dj."updatedAt",
        dj.name,
        dj."salaryType",
        dj."minSalary",
        dj."maxSalary",
        dj."salaryCurrency",
        dj.deadline,
        dj."organizationName",
        dj."infoUrl",
        dj."latvianRegion", -- Added the original array column as requested
        array_to_string(dj."latvianRegion", ', ') AS "latvianRegionString",

        -- Step 2: Custom Column "IrRīga"
        -- This uses a CASE statement to check if 'Rīga' is an element
        -- in the "latvianRegion" array.
        CASE
            WHEN 'RIGA' = ANY(dj."latvianRegion") THEN 'Rīga'
            ELSE 'Ārpus Rīgas'
        END AS "IrRīga",

        -- Step 3: Custom Column "AverageMonthlySalary"
        CASE
            WHEN dj."salaryType" = 'HOURLY_RATE_BRUTTO' AND dj."minS

C:\Users\apmer\AppData\Local\Temp\ipykernel_19556\770523252.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Query executed successfully and data loaded into DataFrame.
Database connection closed.


In [106]:
#get month assignment df for all jobs
query_to_run = """
SELECT
    -- Final selected columns based on "Removed Columns"
    dj.id,
    monthly_series."MonthAssignment"::date,
    
    -- "Added Custom" step to get the month name.
    -- 'TMMonth' is used to get the full month name without extra padding.
    TRIM(TO_CHAR(monthly_series."MonthAssignment", 'TMMonth')) AS "Month"

FROM
    -- "Source" step: Start with the base table
    "DarbnesisJob" dj
    
-- "Expanded MonthAssignments" step:
-- This is the core of the transformation. We use a CROSS JOIN LATERAL
-- with generate_series to create a new row for each month between the
-- start and end dates of the original row.
CROSS JOIN LATERAL
    generate_series(
        -- "StartMonth" logic: DATE_TRUNC('month', ...) gets the first day of the month.
        DATE_TRUNC('month', dj."createdAt")::date,
        
        -- "EndMonth" logic
        DATE_TRUNC('month', dj."deadline")::date,
        
        -- Interval: The step for the series generation.
        '1 month'::interval
        
    ) AS monthly_series("MonthAssignment") -- Alias the generated series table and its column

WHERE
    -- "Filtered Rows" step
    dj."infoUrl" IS NOT NULL AND dj."infoUrl" <> ''
    
    -- It's also good practice to ensure the date range is valid.
    -- generate_series handles this gracefully by returning zero rows if start > end,
    -- but an explicit filter can sometimes improve clarity.
    AND dj."deadline" >= dj."createdAt"

ORDER BY
    dj.id, "MonthAssignment"; -- Optional: ordering for predictable results
"""
df_month_assignments = execute_sql_query(query_to_run)

Connecting to the PostgreSQL database...
Executing query: "
SELECT
    -- Final selected columns based on "Removed Columns"
    dj.id,
    monthly_series."MonthAssignment"::date,

    -- "Added Custom" step to get the month name.
    -- 'TMMonth' is used to get the full month name without extra padding.
    TRIM(TO_CHAR(monthly_series."MonthAssignment", 'TMMonth')) AS "Month"

FROM
    -- "Source" step: Start with the base table
    "DarbnesisJob" dj

-- "Expanded MonthAssignments" step:
-- This is the core of the transformation. We use a CROSS JOIN LATERAL
-- with generate_series to create a new row for each month between the
-- start and end dates of the original row.
CROSS JOIN LATERAL
    generate_series(
        -- "StartMonth" logic: DATE_TRUNC('month', ...) gets the first day of the month.
        DATE_TRUNC('month', dj."createdAt")::date,

        -- "EndMonth" logic
        DATE_TRUNC('month', dj."deadline")::date,

        -- Interval: The step for the series generation.
    

C:\Users\apmer\AppData\Local\Temp\ipykernel_19556\770523252.py:22: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



Query executed successfully and data loaded into DataFrame.
Database connection closed.


In [107]:
#get bonusu df 
file_path = r"C:\Users\apmer\Downloads\big excelis (1)_peters (3).xlsx"
sheet_name = "HUIJA"

try:
    # Step 1, 2, 3, 4: Source, Navigation, Headers, and Type Change for 'id'
    # pd.read_excel handles these steps. We specify the sheet_name and
    # set the 'id' column type to string to match the Power Query logic.
    print(f"Reading data from '{file_path}' sheet '{sheet_name}'...")
    df = pd.read_excel(file_path, sheet_name=sheet_name, dtype={'id': str})
    
    # Define the columns to be removed, as specified in Power Query
    cols_to_remove = ["isco-08 code", "Mātes kompānija"]
    
    # Step 5: Removed Columns
    # Use .drop() to remove the specified columns. We add a check to ensure
    # the columns exist before trying to drop them to avoid errors.
    existing_cols_to_remove = [col for col in cols_to_remove if col in df.columns]
    df_after_remove = df.drop(columns=existing_cols_to_remove)
    print(f"Removed columns: {existing_cols_to_remove}")

    # Step 6: Unpivoted Other Columns
    # This is the key step. We use pandas' melt() function.
    # - id_vars=['id']: This is the "anchor" column that will not be unpivoted.
    # - var_name='Attribute': This is the name for the new column that will hold the names of the old columns.
    # - value_name='Value': This is the name for the new column that will hold the values from the old columns.
    df_bonusi = pd.melt(
        df_after_remove, 
        id_vars=['id'], 
        var_name='Attribute', 
        value_name='Value'
    )
    
    print("\nProcessing complete. 'df_bonusi' DataFrame created.")
    print("\n--- DataFrame Info ---")
    df_bonusi.info()

    print("\n--- DataFrame Head ---")
    print(df_bonusi.head())


except FileNotFoundError:
    print(f"Error: The file was not found at the specified path: {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

#get alias list
# Define the file path to your Excel file.
file_path = r"C:\IT Projekti\Darbnesis Data project\Alias LIST SSOT.xlsx"

try:
    print(f"Reading data from '{file_path}'...")
    
    # [THE FIX] Explicitly define the data type for registration_number as a string.
    df_alias_list = pd.read_excel(file_path, dtype={'registration_number': str})
    
    print("\nProcessing complete. 'df_alias_list' DataFrame has been created with correct types.")
    
    print("\n--- DataFrame Info ---")
    df_alias_list.info()

except FileNotFoundError:
    print(f"Error: The file was not found at the specified path: {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Reading data from 'C:\Users\apmer\Downloads\big excelis (1)_peters (3).xlsx' sheet 'HUIJA'...
Removed columns: ['isco-08 code', 'Mātes kompānija']

Processing complete. 'df_bonusi' DataFrame created.

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19360 entries, 0 to 19359
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   id         19305 non-null  object 
 1   Attribute  19360 non-null  object 
 2   Value      19360 non-null  float64
dtypes: float64(1), object(2)
memory usage: 453.9+ KB

--- DataFrame Head ---
                                     id               Attribute  Value
0  abbb48df-851f-4843-9ddb-2d3b6906a6cf  apmaksats_atvalinajums    0.0
1  3587100c-f83c-4060-9025-9d84c8de29b1  apmaksats_atvalinajums    0.0
2  223b9f88-dc8f-46c5-8e2f-617cf35b24bb  apmaksats_atvalinajums    0.0
3  052b98d0-6b4f-4e17-a4b4-51616f0d55a1  apmaksats_atvalinajums    0.0
4  c8712165-f648-4e1c-ae8e-8d92a85f

In [108]:
# ==============================================================================
# [CORRECTED] Data Cleaning Step for df_alias_list
# ==============================================================================

# [THE FIX] Define the registration numbers as STRINGS to match the DataFrame.
incorrect_reg_num = '10103586000'
correct_reg_num = '50003704731'

# Find a valid row with the correct registration number to use as a template.
# This lookup will now succeed because the types match.
correct_row_template = df_alias_list[df_alias_list['registration_number'] == correct_reg_num].iloc[0]
correct_official_name = correct_row_template['official_name']
correct_nace_codes = correct_row_template['nace_codes']

# Apply the corrections to all rows that have the incorrect registration number.
# We will make them identical to the correct "POLDMA" entry.
df_alias_list.loc[df_alias_list['registration_number'] == incorrect_reg_num, 'registration_number'] = correct_reg_num
df_alias_list.loc[df_alias_list['registration_number'] == incorrect_reg_num, 'official_name'] = correct_official_name
df_alias_list.loc[df_alias_list['registration_number'] == incorrect_reg_num, 'nace_codes'] = correct_nace_codes

print(f"Data corrected for registration number: {incorrect_reg_num}")
print("You must re-run all subsequent cells that create DataFrames from df_alias_list.")

Data corrected for registration number: 10103586000
You must re-run all subsequent cells that create DataFrames from df_alias_list.


In [109]:
#get NACE code df
# Step 1: Select the required columns and create a working copy.
# Also, drop any rows where 'nace_codes' is missing (None/NaN) to prevent errors.
df_nace_codes = df_alias_list[['registration_number', 'nace_codes']].dropna().copy()

# Step 2: Convert the string representation of a list into an actual Python list.
# We use ast.literal_eval() because it's a secure way to parse Python literals.
# Using a simple function with a try-except block makes it robust against malformed strings.
def safe_literal_eval(s):
    try:
        return ast.literal_eval(s)
    except (ValueError, SyntaxError):
        # Return an empty list if the string is not a valid list format
        return []

df_nace_codes['nace_codes'] = df_nace_codes['nace_codes'].apply(safe_literal_eval)

# Step 3: "Unpivot" or "Explode" the list of NACE codes into separate rows.
# The .explode() method duplicates the 'registration_number' for each item in the list.
df_nace_codes = df_nace_codes.explode('nace_codes')

# Step 4: Drop duplicate rows where the combination of registration_number and nace_codes is the same.
df_nace_codes.drop_duplicates(subset=['registration_number', 'nace_codes'], inplace=True)

# Step 5: Add the 'category' column by taking the first 2 digits of the NACE code.
# The .str accessor lets us use string methods like slicing on the entire column.
df_nace_codes['category'] = df_nace_codes['nace_codes'].str[:2]

# Step 6: Reset the index for a clean, final DataFrame.
df_nace_codes.reset_index(drop=True, inplace=True)

print("\n--- 'df_nace_codes' DataFrame created successfully ---")
print("\n--- DataFrame Info ---")
df_nace_codes.info()

print("\n--- Final DataFrame ---")
print(df_nace_codes)


--- 'df_nace_codes' DataFrame created successfully ---

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10558 entries, 0 to 10557
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   registration_number  10558 non-null  object
 1   nace_codes           10558 non-null  object
 2   category             10558 non-null  object
dtypes: object(3)
memory usage: 247.6+ KB

--- Final DataFrame ---
      registration_number nace_codes category
0             41503050998      46.83       46
1             41503050998      46.73       46
2             40003837873      46.71       46
3             40003837873      23.61       23
4             42103081417      64.20       64
...                   ...        ...      ...
10553         40103761688      43.31       43
10554         40203368373      41.20       41
10555         40003995590      25.61       25
10556         40103394007      49.41    

In [110]:
#get cleaned data for salaries and merge into darbnesis jobs
file_path = r"C:\Users\apmer\Downloads\Peteris_algas+ID2 (3).xlsx"
sheet_name = "Sheet2"

try:
    # Step 1 & 2 & 3 & 4: Source, Navigation, Promoted Headers, and initial Changed Type
    # pandas' read_excel can handle these steps in one function call.
    # - "sheet_name" targets the correct sheet.
    # - The first row is automatically used as headers (Promoted Headers).
    # - "dtype={'id': str}" sets the 'id' column to a string type, just like in Power Query.
    print(f"Reading data from '{file_path}' sheet '{sheet_name}'...")
    df = pd.read_excel(file_path, sheet_name=sheet_name, dtype={'id': str})

    # Rename column to be more Python-friendly if desired (optional)
    # df = df.rename(columns={"Vidēja x slodze": "Videja_x_slodze"})
    # original_col_name = "Videja_x_slodze"
    original_col_name = "Vidēja x slodze" # Using original name

    # Step 5 & 6: Removed Errors and Changed Type for "Vidēja x slodze"
    # This is the most direct way to replicate Power Query's "Remove Errors" logic.
    # 1. pd.to_numeric attempts to convert the column to a number.
    # 2. "errors='coerce'" will replace any value that cannot be converted with 'NaN' (Not a Number).
    df[original_col_name] = pd.to_numeric(df[original_col_name], errors='coerce')

    # 3. dropna() removes rows where the specified column ('Vidēja x slodze') is NaN.
    #    This is the equivalent of "Table.RemoveRowsWithErrors".
    df_cleaned_salary = df.dropna(subset=[original_col_name])
    df_cleaned_salary = df_cleaned_salary.rename(columns={"Vidēja x slodze": "new_salary"})

    # The 'Vidēja x slodze' column is now of a numeric type (float64 or int64).
    # You can explicitly cast it if needed, but pd.to_numeric already handles it.
    # df_cleaned_salary[original_col_name] = df_cleaned_salary[original_col_name].astype(float)
    
    print("\nProcessing complete.")
    print("\n--- DataFrame Info ---")
    df_cleaned_salary.info()

    print("\n--- DataFrame Head ---")
    print(df_cleaned_salary.head())


except FileNotFoundError:
    print(f"Error: The file was not found at the specified path: {file_path}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

merged_df = pd.merge(df_darbnesis_job,df_cleaned_salary, on='id', how='left')
merged_df['AverageMonthlySalary'] = np.where(
    merged_df['new_salary'].notna(),    # Condition
    merged_df['new_salary'],            # Value if True
    merged_df['AverageMonthlySalary']   # Value if False
)
df_darbnesis_job = merged_df.drop(columns=['new_salary'])

Reading data from 'C:\Users\apmer\Downloads\Peteris_algas+ID2 (3).xlsx' sheet 'Sheet2'...

Processing complete.

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
Index: 342 entries, 0 to 349
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          342 non-null    object 
 1   new_salary  342 non-null    float64
dtypes: float64(1), object(1)
memory usage: 8.0+ KB

--- DataFrame Head ---
                                     id  new_salary
0  79060d8d-d6ea-478d-ab12-687c5e4ec50d      1200.0
1  e12026ce-77ca-4ff2-a998-faeeea322fbe      1200.0
2  adcaba82-19e5-4b22-98b5-168788d2d230      1200.0
3  940be8ac-2dcf-4ae8-8cda-2b36f543b8d5      1200.0
4  ff83ceb3-476f-4cd2-a50c-5d990ff63763      1200.0


In [111]:
#get isco codes added to darbnesis job
# Define the file path and the specific Excel Table name from your script
file_path = r"C:\Users\apmer\Downloads\Apranga konkurenti-īstie ISCO kodi (1).xlsx"
table_name = "Aprangas konkurentu vakances"

try:
    # This combines all the Power Query steps into one efficient operation.
    #
    # - sheet_name=table_name: pandas' read_excel can read from a named Excel Table
    #   if you pass the table name as the sheet_name.
    # - usecols=["id", "Isco kods"]: This is the direct, efficient equivalent of
    #   "Table.SelectColumns". It only reads these two columns into memory.
    # - dtype={'id': str, 'Isco kods': 'Int64'}: This replicates "Changed Type".
    #   'id' is set to string. 'Int64' (capital 'I') is pandas' nullable integer type,
    #   which handles potential blank cells in the numeric column gracefully,
    #   just like Power Query.
    
    print(f"Reading 'id' and 'Isco kods' columns from '{table_name}' in '{file_path}'...")
    
    df_istie_isco_kodi = pd.read_excel(
        file_path,
        sheet_name=table_name,
        usecols=["id", "Isco kods"],
        dtype={'id': str, 'Isco kods': 'Int64'}
    )
    
    print("\nProcessing complete. 'df_istie_isco_kodi' DataFrame created.")
    print("\n--- DataFrame Info ---")
    df_istie_isco_kodi.info()

    print("\n--- DataFrame Head ---")
    print(df_istie_isco_kodi.head())

except FileNotFoundError:
    print(f"Error: The file was not found at the specified path: {file_path}")
except ValueError as e:
    # This error often happens if the sheet/table name is not found
    print(f"Error reading the Excel file. Please check if a table or sheet named '{table_name}' exists.")
    print(f"Details: {e}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

# The final DataFrame is 'df_istie_isco_kodi'

df_darbnesis_job = pd.merge(
    df_darbnesis_job,
    df_istie_isco_kodi,
    on='id',
    how='left'
)

Reading 'id' and 'Isco kods' columns from 'Aprangas konkurentu vakances' in 'C:\Users\apmer\Downloads\Apranga konkurenti-īstie ISCO kodi (1).xlsx'...

Processing complete. 'df_istie_isco_kodi' DataFrame created.

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         350 non-null    object
 1   Isco kods  350 non-null    Int64 
dtypes: Int64(1), object(1)
memory usage: 5.9+ KB

--- DataFrame Head ---
                                     id  Isco kods
0  79060d8d-d6ea-478d-ab12-687c5e4ec50d       5223
1  e12026ce-77ca-4ff2-a998-faeeea322fbe       5223
2  adcaba82-19e5-4b22-98b5-168788d2d230       5223
3  940be8ac-2dcf-4ae8-8cda-2b36f543b8d5       5223
4  ff83ceb3-476f-4cd2-a50c-5d990ff63763       5223


In [112]:
#get distinct org df
df_distinct_orgs = df_alias_list[['registration_number', 'official_name']] \
    .drop_duplicates(subset=['registration_number']) \
    .reset_index(drop=True) \
    .copy()


print("\n--- 'df_distinct_orgs' DataFrame created successfully ---")

# Display info and the final result to verify
print("\n--- DataFrame Info ---")
df_distinct_orgs.info()

print("\n--- Final DataFrame ---")
print(df_distinct_orgs)

print(f"\nTotal unique organizations found: {len(df_distinct_orgs)}")


--- 'df_distinct_orgs' DataFrame created successfully ---

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6578 entries, 0 to 6577
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   registration_number  6577 non-null   object
 1   official_name        6577 non-null   object
dtypes: object(2)
memory usage: 102.9+ KB

--- Final DataFrame ---
     registration_number                                      official_name
0            41503050998        Sabiedrība ar ierobežotu atbildību "VENTUM"
1            40003837873                             SIA "Plieņa dzirnavas"
2            42103081417                                     AS Longo Group
3            40900040782          Ogres novada Kultūras un tūrisma pārvalde
4            40203506125  Sabiedrība ar ierobežotu atbildību "KK OZOLNIEKI"
...                  ...                                                ...
6573         402

In [113]:
#add registration number column to df darbnesis job
lookup_df = df_alias_list[['organizationName', 'registration_number']].drop_duplicates(subset=['organizationName'])

print("\n--- Prepared Lookup Table (from df_alias_list) ---")
print(lookup_df)

# --- Step 2: Perform the left merge and overwrite df_darbnesis_job ---
# We merge df_darbnesis_job with our clean lookup_df.
# - on='organizationName': This is the column to match on.
# - how='left': This keeps all rows from df_darbnesis_job and adds data from lookup_df where matches are found.
df_darbnesis_job = pd.merge(
    df_darbnesis_job,
    lookup_df,
    on='organizationName',
    how='left'
)


--- Prepared Lookup Table (from df_alias_list) ---
                                organizationName registration_number
0                                     Ventum SIA         41503050998
1                           SIA Pliena dzirnavas         40003837873
2                                Longo Group, AS         42103081417
3      Ogres novada Kultūras un tūrisma pārvalde         40900040782
4                                   Kk ozolnieki         40203506125
...                                          ...                 ...
17174                                SIA Elmotem                 NaN
17176                                 SIA Rakfor                 NaN
17185                                 Sia Strela                 NaN
17197                 Riga Golf Innovations, SIA                 NaN
17198                             Sportland, SIA         40003530961

[10256 rows x 2 columns]


In [114]:
#add konkurentu df
csv_file_path = r"C:\IT Projekti\Darbnesis Data project\Aprangas konkurentu vakances.csv"

# The constant registration number for your company
aprangas_reg_number = '40003672631'

try:
    # --- Step 1: Get the list of competitor IDs from the CSV file ---
    # We only need the 'id' column to act as our filter.
    # Reading it as a string ('str') ensures it will match the 'id' type in df_darbnesis_job.
    print(f"Reading competitor IDs from '{csv_file_path}'...")
    df_konkurentu_ids = pd.read_csv(csv_file_path, usecols=['id'], dtype={'id': str}, delimiter=';')

    # --- Step 2: Filter df_darbnesis_job to keep only the competitor rows ---
    # An 'inner' merge is the most direct way to do this. It keeps only the rows
    # where the 'id' exists in both df_darbnesis_job and df_konkurentu_ids.
    df_filtered_competitors = pd.merge(
        df_darbnesis_job,
        df_konkurentu_ids,
        on='id',
        how='inner'
    )
    print("\nFiltered df_darbnesis_job to keep only competitor vacancies.")

    # --- Step 3: Get the distinct list of competitor registration numbers ---
    # From the filtered DataFrame, we select the 'registration_number' column,
    # drop any missing values (NaN), and then get the unique values.
    competitor_reg_numbers = df_filtered_competitors['registration_number'].dropna().unique()
    print(f"Found {len(competitor_reg_numbers)} unique competitor registration numbers.")

    # --- Step 4: Construct the final DataFrame ---
    # Create a new DataFrame with the specified structure.
    # Pandas will automatically "broadcast" the single 'aprangas_reg_number' value
    # to all rows, matching the length of the 'competitor_reg_numbers' list.
    df_kompanijas_konkurenti = pd.DataFrame({
        'Registration number': aprangas_reg_number,
        'Competitor registration number': competitor_reg_numbers
    })

    print("\n--- 'df_kompanijas_konkurenti' DataFrame created successfully ---")
    print("\n--- DataFrame Info ---")
    df_kompanijas_konkurenti.info()
    
    print("\n--- Final DataFrame ---")
    print(df_kompanijas_konkurenti)

except FileNotFoundError:
    print(f"Error: The file was not found at '{csv_file_path}'. Please check the path.")
except KeyError:
    print(f"Error: The CSV file at '{csv_file_path}' does not contain an 'id' column.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Reading competitor IDs from 'C:\IT Projekti\Darbnesis Data project\Aprangas konkurentu vakances.csv'...

Filtered df_darbnesis_job to keep only competitor vacancies.
Found 11 unique competitor registration numbers.

--- 'df_kompanijas_konkurenti' DataFrame created successfully ---

--- DataFrame Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11 entries, 0 to 10
Data columns (total 2 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Registration number             11 non-null     object
 1   Competitor registration number  11 non-null     object
dtypes: object(2)
memory usage: 308.0+ bytes

--- Final DataFrame ---
   Registration number Competitor registration number
0          40003672631                    40003604688
1          40003672631                    40003530961
2          40003672631                    40003398680
3          40003672631                    40003813061
4          4

## we define chart as class object and make helper functions for data manipulation we define some theme style guide

In [115]:
#define helpter functions for highlighting and wrapping
def prepare_data_with_highlight(df, full_color_palette, selected_company, highlight_color):
    df_copy = df.copy()
    other_colors = [color for color in full_color_palette if color != highlight_color]
    random.shuffle(other_colors)
    color_assignment = {}
    other_color_index = 0
    for company in df_copy.index:
        if company == selected_company:
            color_assignment[company] = highlight_color
        else:
            color_assignment[company] = other_colors[other_color_index % len(other_colors)]
            other_color_index += 1
    df_copy['barcolor'] = df_copy.index.map(color_assignment)
    return df_copy

def wrap_df_index_labels(
    df: pd.DataFrame,
    usable_chart_width: int = 500,
    approx_char_pixel_width: int = 7,
    padding_per_bar: int = 15,
    min_wrap_limit: int = 12
) -> pd.DataFrame:
    """
    Dynamically wraps the index labels of a DataFrame to fit in a chart.

    This function calculates the optimal character limit for wrapping text labels
    based on the number of data points and the available chart width. It then
    applies this wrapping to the DataFrame's index, replacing newlines with
    HTML line breaks (<br>) for chart rendering (e.g., in Plotly).

    Args:
        df (pd.DataFrame): The input DataFrame whose index labels need wrapping.
                           The function creates a copy and does not modify the original.
        usable_chart_width (int, optional): The approximate usable pixel width
                                             of the chart's plotting area. Defaults to 500.
        approx_char_pixel_width (int, optional): An estimate of the average pixel
                                                 width of a single character in the font used.
                                                 Defaults to 7 (a good estimate for a 14px font).
        padding_per_bar (int, optional): The amount of pixel padding to subtract
                                         for each bar/category in the chart. Defaults to 15.
        min_wrap_limit (int, optional): The minimum character limit to fall back on,
                                        preventing labels from becoming too narrow.
                                        Defaults to 12.

    Returns:
        pd.DataFrame: A new DataFrame with the index labels formatted and wrapped.
    """
    # Work on a copy to avoid modifying the original DataFrame
    prepared_df = df.copy()

    # --- 1. Calculate the dynamic character limit for wrapping ---
    number_of_bars = len(prepared_df.index)
    if number_of_bars == 0:
        return prepared_df # Return early if the DataFrame is empty

    # Calculate available space for each label
    space_per_bar = (usable_chart_width / number_of_bars) - padding_per_bar

    # Calculate character limit based on space, with a minimum fallback
    dynamic_char_limit = max(
        min_wrap_limit, int(space_per_bar / approx_char_pixel_width)
    )
    print(f"Dynamically calculated character limit for wrapping: {dynamic_char_limit}")

    # --- 2. Apply wrapping to the DataFrame index ---
    formatted_labels = [
        textwrap.fill(label, width=dynamic_char_limit).replace('\n', '<br>')
        for label in prepared_df.index
    ]
    prepared_df.index = formatted_labels

    return prepared_df



In [116]:
# ==============================================================================
# [UPGRADED] Reusable function to prepare trend data
# ==============================================================================
def prepare_trend_data(
    base_jobs_df,
    monthly_assignments_df,
    grouping_cols,
    aggregation_mode='count',
    aggregation_col=None,
    y_col_name='Value',
    category_col_name=None,
    nace_category=None, # Optional: for broad filtering
    specific_nace_codes=None # Optional: for specific filtering
):
    """
    [UPGRADED] Prepares trend data. Can filter by broad NACE category OR a
    specific list of NACE codes.
    """
    print(f"--- Preparing data for aggregation: mode='{aggregation_mode}', group_by={grouping_cols} ---")
    
    # --- Step 1: Dynamic NACE Filtering ---
    if specific_nace_codes:
        print(f"Filtering on specific NACE codes: {specific_nace_codes}")
        # Find all registration numbers that have AT LEAST ONE of the specified codes
        sector_reg_numbers = df_nace_codes[df_nace_codes['nace_codes'].isin(specific_nace_codes)]['registration_number'].unique()
    elif nace_category:
        print(f"Filtering on NACE category: {nace_category}")
        sector_reg_numbers = df_nace_codes[df_nace_codes['category'] == nace_category]['registration_number'].unique()
    else:
        print("Error: Must provide either 'nace_category' or 'specific_nace_codes'.")
        return pd.DataFrame()

    # --- Step 2: Filter base jobs and Merge ---
    cols_to_select = ['id'] + [col for col in grouping_cols if col in base_jobs_df.columns]
    if aggregation_col and aggregation_col not in cols_to_select:
        cols_to_select.append(aggregation_col)
    sector_jobs_df = base_jobs_df[base_jobs_df['registration_number'].isin(sector_reg_numbers)][cols_to_select].copy()
    
    cleaned_assignments = monthly_assignments_df.dropna(subset=['Month', 'id', 'MonthAssignment'])
    sector_jobs_with_months = pd.merge(sector_jobs_df, cleaned_assignments, on='id', how='left')

    # --- Step 3: Date Filtering ---
    sector_jobs_with_months['MonthAssignment'] = pd.to_datetime(sector_jobs_with_months['MonthAssignment'])
    today = pd.to_datetime('today')
    end_date = pd.Timestamp(today.year, today.month, 1)
    start_date = end_date - pd.DateOffset(months=3)
    last_3_months_data = sector_jobs_with_months[
        (sector_jobs_with_months['MonthAssignment'] >= start_date) &
        (sector_jobs_with_months['MonthAssignment'] < end_date)
    ]

    # --- Step 4: Aggregation ---
    if aggregation_mode == 'count':
        aggregated_data = last_3_months_data.groupby(grouping_cols).size().reset_index(name=y_col_name)
    elif aggregation_mode == 'mean' and aggregation_col:
        aggregated_data = last_3_months_data.groupby(grouping_cols)[aggregation_col].mean().reset_index(name=y_col_name)
    else:
        return pd.DataFrame()

    # --- Step 5: Final Data Prep (Translate & Sort) ---
    aggregated_data = aggregated_data.rename(columns={'Month': 'Mēnesis'})
    if category_col_name and len(grouping_cols) > 1:
        aggregated_data = aggregated_data.rename(columns={grouping_cols[1]: category_col_name})
    
    month_translation = {
        'January': 'Janvāris', 'February': 'Februāris', 'March': 'Marts',
        'April': 'Aprīlis', 'May': 'Maijs', 'June': 'Jūnijs',
        'July': 'Jūlijs', 'August': 'Augusts', 'September': 'Septembris',
        'October': 'Oktobris', 'November': 'Novembris', 'December': 'Decembris'
    }
    aggregated_data['Mēnesis'] = aggregated_data['Mēnesis'].map(month_translation)
    
    latvian_month_order = list(month_translation.values())
    aggregated_data['Mēnesis'] = pd.Categorical(aggregated_data['Mēnesis'], categories=latvian_month_order, ordered=True)
    aggregated_data = aggregated_data.dropna(subset=['Mēnesis']).sort_values('Mēnesis')
    
    print("Data preparation complete.\n")
    return aggregated_data

In [117]:
# ==============================================================================
# FINAL, ROBUST ReportBlock CLASS (with Themed Pie Chart)
# ==============================================================================
from reportlab.platypus import Table, TableStyle
from reportlab.lib import colors
import pandas as pd
import plotly.graph_objects as go

class ReportBlock:
    """
    [FINAL & ROBUST] The constructor is flexible. The pie chart method
    has been corrected to use the report's custom color palette.
    """
    def __init__(self, usable_width, block_title=None, block_title_style=None, description_text=None, description_style=None, layout='vertical'):
        self.usable_width = usable_width
        self.block_title = block_title
        self.block_title_style = block_title_style
        self.description_text = description_text
        self.description_style = description_style
        self.layout = layout

    # ... (all other methods remain unchanged) ...
    def _create_bar_chart(self, width, height, axis_format='raw'):
        y_axis_title = self.chart_data.columns[0]
        x_axis_title = self.chart_data.index.name or ''
        max_val = self.chart_data[y_axis_title].max()
        label_threshold = max_val * 0.15
        df_inside = self.chart_data[self.chart_data[y_axis_title] >= label_threshold]
        df_outside = self.chart_data[self.chart_data[y_axis_title] < label_threshold]
        y_axis_config = dict(
            title_text=f"<b>{y_axis_title}</b>", title_font=dict(family="Ubuntu", size=30),
            tickfont=dict(family="Ubuntu", size=16), gridcolor='#C0C0C0', griddash='6px,6px', gridwidth=2,
            showgrid=True, showline=False, ticks='', rangemode='tozero',
            range=[0, max_val * 1.15]
        )
        chart_layout = go.Layout(
            font=dict(family="Ubuntu", color="#404040"), plot_bgcolor='white', paper_bgcolor='white',
            showlegend=False, barmode='overlay',
            margin=dict(l=80, r=20, t=20, b=150),
            xaxis=dict(
                title_text=f"<b>{x_axis_title}</b>", title_font=dict(family="Ubuntu", size=30), title_standoff=25,
                tickfont=dict(family="Ubuntu", size=14), showgrid=False, showline=False,
                ticks='', tickangle=0, automargin=True
            ),
            yaxis=y_axis_config
        )
        fig = go.Figure(layout=chart_layout)
        fig.add_trace(go.Bar(
            x=df_inside.index, y=df_inside[y_axis_title],
            marker=dict(color=df_inside['barcolor'], cornerradius=10),
            texttemplate='<b>%{y}</b>', textposition='inside',
            insidetextfont=dict(family='Ubuntu', size=24, color='white'),
            insidetextanchor='end', constraintext='inside'
        ))
        fig.add_trace(go.Bar(
            x=df_outside.index, y=df_outside[y_axis_title],
            marker=dict(color=df_outside['barcolor'], cornerradius=10),
            texttemplate='<b>%{y}</b>', textposition='outside',
            outsidetextfont=dict(family='Ubuntu', size=24, color='#404040'),
            cliponaxis=False
        ))
        img_bytes = fig.to_image(format="png", width=width*3, height=height*3, scale=3)
        return Image(BytesIO(img_bytes), width=width, height=height)

    # [THE FIX] This method now accepts a 'colors' argument.
    def _create_pie_chart(self, chart_data, colors, width, height):
        """ Creates a pie chart. chart_data is a Pandas Series. """
        fig = go.Figure(data=[go.Pie(
            labels=chart_data.index,
            values=chart_data.values,
            textinfo='value+percent',
            textposition='outside',
            hole=.3
        )])
        fig.update_traces(
            textfont_size=14,
            # [THE FIX] Applying the custom color palette to the slices.
            marker=dict(colors=colors, line=dict(color='#FFFFFF', width=2))
        )
        fig.update_layout(
            title_text='Vakanču skaits pēc pozīcijām',
            title_font=dict(size=16, family="Ubuntu"),
            legend_title_text='Isco-08 code',
            plot_bgcolor='white',
            paper_bgcolor='white',
            margin=dict(l=20, r=20, t=50, b=20),
            uniformtext_minsize=12, 
            uniformtext_mode='hide'
        )
        img_bytes = fig.to_image(format="png", width=width*3, height=height*3, scale=3)
        return Image(BytesIO(img_bytes), width=width, height=height)

    def _create_line_chart(self, chart_data, x_col, y_col, category_col, selected_company, highlight_color, color_palette, x_axis_title, y_axis_title, show_legend, width, height):
        fig = go.Figure()
        other_colors = [color for color in color_palette if color != highlight_color]
        random.shuffle(other_colors)
        color_idx = 0
        for category_name in chart_data[category_col].unique():
            if category_name == selected_company:
                line_color = highlight_color
                line_width = 16
            else:
                line_color = other_colors[color_idx % len(other_colors)]
                line_width = 12
                color_idx += 1
            category_df = chart_data[chart_data[category_col] == category_name]
            fig.add_trace(go.Scatter(
                x=category_df[x_col], y=category_df[y_col], name=category_name,
                mode='lines+markers', line=dict(color=line_color, width=line_width),
                marker=dict(size=line_width)
            ))
        fig.update_layout(
            font=dict(family="Ubuntu", color="#404040"), plot_bgcolor='white', paper_bgcolor='white',
            showlegend=show_legend,
            legend=dict(x=0.02, y=0.98, bgcolor='rgba(255,255,255,0.6)', font=dict(size=24, family="Ubuntu")),
            margin=dict(l=80, r=20, t=20, b=90),
            xaxis=dict(title_text=f"<b>{x_axis_title}</b>", title_font=dict(size=30, family="Ubuntu"), tickfont=dict(size=24, family="Ubuntu"), showgrid=False),
            yaxis=dict(title_text=f"<b>{y_axis_title}</b>", title_font=dict(size=30, family="Ubuntu"), tickfont=dict(size=24, family="Ubuntu"), gridcolor='#C0C0C0', griddash='6px,6px', gridwidth=2, rangemode='tozero')
        )
        img_bytes = fig.to_image(format="png", width=width*3, height=height*3, scale=3)
        return Image(BytesIO(img_bytes), width=width, height=height)

    def _create_matrix_table(self, chart_data, index_col, columns_col, values_col):
        matrix_df = chart_data.pivot_table(index=index_col, columns=columns_col, values=values_col, aggfunc='nunique').fillna(0).astype(int)
        total_uniques = chart_data.groupby(index_col)[values_col].nunique()
        total_uniques.name = 'Total'
        final_matrix_df = matrix_df.join(total_uniques)
        month_order = ['April', 'May', 'June']
        present_months = [col for col in final_matrix_df.columns if col in month_order]
        sorted_months = sorted(present_months, key=lambda m: month_order.index(m))
        final_matrix_df = final_matrix_df[sorted_months + ['Total']]
        final_matrix_df = final_matrix_df.sort_values(by='Total', ascending=False)
        total_row = final_matrix_df.sum().to_frame().T
        total_row.index = ['Total']
        final_matrix_df = pd.concat([final_matrix_df, total_row])
        header = ['Mātes kompānija'] + [str(col) for col in final_matrix_df.columns]
        data_for_table = [header] + [[index] + row.tolist() for index, row in final_matrix_df.iterrows()]
        table = Table(data_for_table, hAlign='LEFT')
        table_style = TableStyle([
            ('FONTNAME', (0, 0), (-1, -1), 'Ubuntu'), ('FONTSIZE', (0, 0), (-1, -1), 9),
            ('VALIGN', (0, 0), (-1, -1), 'MIDDLE'), ('GRID', (0, 0), (-1, -1), 0.5, colors.lightgrey),
            ('FONTNAME', (0, 0), (-1, 0), 'Ubuntu-Bold'), ('LINEBELOW', (0, 0), (-1, 0), 1.5, colors.HexColor("#05a1b4")),
            ('ALIGN', (1, 0), (-1, 0), 'CENTER'), ('FONTNAME', (0, -1), (-1, -1), 'Ubuntu-Bold'),
            ('BACKGROUND', (0, -1), (-1, -1), colors.whitesmoke), ('LINEABOVE', (0, -1), (-1, -1), 1.5, colors.darkgrey),
            ('FONTNAME', (-1, 1), (-1, -1), 'Ubuntu-Bold'), ('ALIGN', (0, 0), (0, -1), 'LEFT'),
            ('RIGHTPADDING', (0, 0), (0, -1), 12), ('ALIGN', (1, 1), (-1, -1), 'CENTER'),
        ])
        for i in range(1, len(final_matrix_df)):
            if i % 2 == 0:
                table_style.add('BACKGROUND', (0, i), (-1, i), colors.HexColor("#f0f0f0"))
        table.setStyle(table_style)
        return table

    def _create_ribbon_chart(self, chart_data, colors, width, height):
        fig = go.Figure()
        for company in chart_data.index:
            fig.add_trace(go.Scatter(
                x=chart_data.columns, y=chart_data.loc[company], name=company,
                mode='lines', line=dict(width=0.5, color='white'),
                stackgroup='one', fillcolor=colors.get(company, '#cccccc')
            ))
        fig.update_layout(
            title_text='Vakanču skaits', title_font=dict(size=16, family="Ubuntu"),
            showlegend=True, legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
            plot_bgcolor='white', paper_bgcolor='white', xaxis=dict(showgrid=False),
            yaxis=dict(title_text='Count of id', type='linear', showgrid=True, gridcolor='#e0e0e0'),
            margin=dict(l=50, r=20, t=50, b=20)
        )
        img_bytes = fig.to_image(format="png", width=width*3, height=height*3, scale=3)
        return Image(BytesIO(img_bytes), width=width, height=height)
    
    def render(self, chart_type, **kwargs):
        elements = []
        if self.block_title and self.block_title_style:
            elements.append(Paragraph(self.block_title, self.block_title_style))
            elements.append(Spacer(1, 6))
        visual_element = None
        if chart_type == 'bar':
            width_percentage = kwargs.pop('width_percentage', 0.98)
            image_width = self.usable_width * width_percentage
            aspect_ratio = kwargs.pop('aspect_ratio', 0.40)
            image_height = image_width * aspect_ratio
            visual_element = self._create_bar_chart(width=image_width, height=image_height, **kwargs)
        elif chart_type == 'line':
            width_percentage = kwargs.pop('width_percentage', 0.98)
            image_width = self.usable_width * width_percentage
            aspect_ratio = kwargs.pop('aspect_ratio', 0.6)
            image_height = image_width * aspect_ratio
            visual_element = self._create_line_chart(width=image_width, height=image_height, **kwargs)
        elif chart_type == 'matrix':
            visual_element = self._create_matrix_table(**kwargs)
        elif chart_type == 'ribbon':
            width_percentage = kwargs.pop('width_percentage', 0.98)
            image_width = self.usable_width * width_percentage
            aspect_ratio = 0.6
            image_height = image_width * aspect_ratio
            visual_element = self._create_ribbon_chart(width=image_width, height=image_height, **kwargs)
        elif chart_type == 'pie':
            width_percentage = kwargs.pop('width_percentage', 0.98)
            image_width = self.usable_width * width_percentage
            aspect_ratio = 0.8
            image_height = image_width * aspect_ratio
            visual_element = self._create_pie_chart(width=image_width, height=image_height, **kwargs)

        if not visual_element: return []
        if self.layout == 'horizontal' and self.description_text:
            text_element = Paragraph(self.description_text, self.description_style)
            container_table = Table(
                [[text_element, visual_element]],
                colWidths=[self.usable_width * 0.35, self.usable_width * 0.65],
                hAlign='LEFT'
            )
            container_table.setStyle(TableStyle([('VALIGN', (0,0), (-1,-1), 'TOP')]))
            elements.append(container_table)
        else:
            elements.append(visual_element)
            if self.description_text:
                elements.append(Spacer(1, 6))
                elements.append(Paragraph(self.description_text, self.description_style))
        elements.append(Spacer(1, 18))
        return [KeepTogether(elements)]

    def render_bar_chart(self, chart_data, **kwargs):
        self.chart_data = chart_data
        return self.render('bar', **kwargs)
    def render_line_chart(self, **kwargs):
        return self.render('line', **kwargs)
    def render_matrix_table(self, **kwargs):
        return self.render('matrix', **kwargs)
    def render_ribbon_chart(self, **kwargs):
        return self.render('ribbon', **kwargs)
    def render_pie_chart(self, **kwargs):
        return self.render('pie', **kwargs)

In [118]:
# --- HIGHLIGHT FIRST ---
df_distinct_orgs = prepare_data_with_highlight(
    df=df_distinct_orgs,
    full_color_palette=COLOR_PALETTE,
    selected_company=selected_company,
    highlight_color=HIGHLIGHT_COLOR
)

In [119]:
# ==============================================================================
# CENTRALIZED REPORT STYLING AND SETUP (with separate chart title style)
# ==============================================================================

# --- 1. Font Registration & Family Definitions ---
try:
    pdfmetrics.registerFont(TTFont('Roboto', ROBOTO_REGULAR_PATH))
    pdfmetrics.registerFont(TTFont('Roboto-Bold', ROBOTO_BOLD_PATH))
    pdfmetrics.registerFont(TTFont('Ubuntu', UBUNTU_REGULAR_PATH))
    pdfmetrics.registerFont(TTFont('Ubuntu-Bold', UBUNTU_BOLD_PATH))
    pdfmetrics.registerFontFamily('Roboto', normal='Roboto', bold='Roboto-Bold')
    pdfmetrics.registerFontFamily('Ubuntu', normal='Ubuntu', bold='Ubuntu-Bold')
    print("All fonts and font families registered successfully.")
    main_font = 'Ubuntu'
    main_font_bold = 'Ubuntu-Bold'
except Exception as e:
    print(f"Error registering fonts, defaulting to Roboto. Error: {e}")
    pdfmetrics.registerFontFamily('Roboto', normal='Roboto', bold='Roboto-Bold')
    main_font = 'Roboto'
    main_font_bold = 'Roboto-Bold'

# --- 2. Define All Paragraph Styles ---
styles = getSampleStyleSheet()

# Style for colored main section titles (e.g., "1. Ievads")
intro_title_style = ParagraphStyle(
    'introTitle', parent=styles['h2'], fontName=main_font_bold,
    fontSize=24, leading=28, spaceAfter=12, textColor='#05a1b4'
)

# Style for TEXT sub-section titles (e.g., "3.1. ...")
subsection_title_style = ParagraphStyle(
    'subsectionTitle', parent=styles['h3'], fontName=main_font_bold,
    fontSize=18,      # Reverted to 18pt
    leading=22,       # Reverted to 22pt
    spaceAfter=6,
    textColor='#1a686c'
)

# [NEW] Style specifically for CHART titles (e.g., "Vakanču skaits")
chart_title_style = ParagraphStyle(
    'chartTitle', parent=styles['h3'], fontName=main_font_bold,
    fontSize=14,      # Set to 16pt as requested
    leading=20,
    spaceAfter=8,
    textColor='#1a686c'
)

# Style for all standard body text
intro_body_style = ParagraphStyle(
    'introBody', parent=styles['BodyText'], fontName=main_font,
    fontSize=11, leading=15, spaceAfter=12
)

# Style for bolded sub-headings within body text
sub_heading_style = ParagraphStyle(
    'subHeading', parent=intro_body_style, fontName=main_font_bold, spaceAfter=6
)

# Style for indented bullet points
bullet_style = ParagraphStyle(
    'bulletStyle', parent=intro_body_style, leftIndent=20, spaceAfter=6
)

# --- 3. Initialize the Document Story ---
story = []

print("Report styles defined. Created separate style for chart titles.")

All fonts and font families registered successfully.
Report styles defined. Created separate style for chart titles.


## we build the story

In [120]:
#Ievads


title_text = "1. Ievads"
story.append(Paragraph(title_text, intro_title_style))

# Add the body text, transcribed from the image
body_text_p1 = """
Šī atskaite sniedz uz datiem balstītu pārskatu par darba tirgus situāciju Latvijas mazumtirdzniecības
nozarē, īpašu uzmanību pievēršot SIA “Apranga” vakanču aktivitātei. Salīdzinājumam analizētas arī
galveno konkurentu – LPP Latvia, H&M Hennes & Mauritz SIA un Danija – publicētās vakances pēdējo
trīs mēnešu laikā.
"""
story.append(Paragraph(body_text_p1, intro_body_style))

body_text_p2 = """
Atskaitē iekļauta vakanču dinamika dažādos reģionos, aktuālākie amata veidi un prasības, kā arī
sezonālās tendences un kandidātu meklēšanas paradumi. Detalizēti apskatīts arī atalgojuma un
bonusu salīdzinājums starp uzņēmumiem.
"""
story.append(Paragraph(body_text_p2, intro_body_style))

body_text_p3 = """
Analīze balstīta uz datiem no darba portāla darbe.lv, un to sagatavojis uzņēmums SIA “Visasiespējas”.
"""
story.append(Paragraph(body_text_p3, intro_body_style))

body_text_p4 = """
<b>Mērķis</b> – sniegt SIA “Apranga” praktiskas rekomendācijas, kas palīdzētu uzlabot personāla
piesaistes efektivitāti un konkurētspēju darba tirgū.
"""
story.append(Paragraph(body_text_p4, intro_body_style))

# Add a larger space before the next section
story.append(Spacer(1, 24))

print("Introduction section has been added to the story.")

Introduction section has been added to the story.


In [121]:
# --- 2. KOPSAVILKUMS (SUMMARY) ---

# Add the main title for the section
section_title_2 = "2. Kopsavilkums"
story.append(Paragraph(section_title_2, intro_title_style))

# Add the first main paragraph
summary_p1 = """
Šī atskaite sniedz padziļinātu analīzi par darba tirgus situāciju Latvijas apģērbu mazumtirdzniecības
nozarē, ar uzsvaru uz SIA “Apranga” konkurētspēju personāla piesaistes jomā. Analīze balstās uz
2025. gada aprīļa–jūnija periodā apkopotiem datiem no darba portāla Darbe.lv un ietver galveno
konkurentu – LPP Latvia, H&M, Danija u.c. – salīdzinājumu.
"""
story.append(Paragraph(summary_p1, intro_body_style))

# Add the sub-heading for the key findings
summary_sub_heading = "Galvenie secinājumi:"
story.append(Paragraph(summary_sub_heading, sub_heading_style))

# Add the bullet points
bullet_points = [
    "SIA “Apranga” saglabā stabilu pozīciju darba tirgū ar konkurētspējīgu vakanču skaitu un atalgojumu, pārsniedzot nozares vidējo līmeni.",
    "Vakances koncentrētas Rīgā, kas atbilst uzņēmuma pašreizējam veikalu izvietojumam. Taču konkurenti aktīvāk darbojas arī reģionos, piedāvājot plašākas iespējas kandidātiem ārpus galvaspilsētas.",
    "Populārākā amata pozīcija visā nozarē ir pārdevējs veikalā (ISCO-08 kods 5223), kam seko veikalu vadītāji.",
    "Apranga izceļas ar konsekventu elastīgā darba grafika un veselības apdrošināšanas piedāvājumu, taču sludinājumos trūkst komunikācijas par citiem populāriem bonusiem (piemēram, prēmijām, apmaksātām pusdienām, darbinieku saliedēšanas pasākumiem)..."
]

for point in bullet_points:
    # Prepend the bullet character to the text
    bullet_paragraph = Paragraph(f"• {point}", bullet_style)
    story.append(bullet_paragraph)

# Add the final concluding paragraph
summary_p2 = """
<b>Balstoties uz iegūtajiem datiem, sniegtas praktiskas rekomendācijas personāla piesaistes uzlabošanai,</b> tostarp uzlabot sludinājumu saturu ar pilnīgāku bonusu aprakstu, diferencēt komunikāciju pret studentu auditoriju, un apsvērt referāļu programmas ieviešanu.
"""
story.append(Paragraph(summary_p2, intro_body_style))

# Add a larger space before the next section
story.append(Spacer(1, 24))

print("Summary section has been added to the story.")

Summary section has been added to the story.


In [122]:
# --- 3. VISPĀRĒJAIS TIRGUS APSKATS (GENERAL MARKET OVERVIEW) ---

# Add the main title for the section
section_title_3 = "3. Vispārējais tirgus apskats"
story.append(Paragraph(section_title_3, intro_title_style))

# Add the introductory paragraph for this section
market_overview_p1 = """
Šī sadaļa sniedz pārskatu par mazumtirdzniecības darba tirgus attīstību Latvijā pēdējo trīs mēnešu
laikā, izceļot būtiskākās izmaiņas un aktuālās tendences. Analīze ir strukturēta divās daļās:
"""
story.append(Paragraph(market_overview_p1, intro_body_style))

# Add the first sub-section (3.1)
# Using <b> and <i> tags for inline formatting
market_overview_p2 = """
<b>3.1. daļa</b> aplūko mazumtirdzniecības nozari kopumā, koncentrējoties uz vakancēm, kas atbilst NACE
klasifikācijas kodam <b>47 “Mazumtirdzniecība, izņemot automobiļus un motociklus”</b>. Šajā kategorijā
ietilpst, piemēram, tādi uzņēmumi kā <i>Maxima, Circle K</i> un <i>Apranga</i>.
"""
story.append(Paragraph(market_overview_p2, intro_body_style))

# Add a small spacer for visual separation before the next part
story.append(Spacer(1, 12))

# Add the second sub-section (3.2)
market_overview_p3 = """
<b>3.2. daļa</b> fokusējas uz apģērbu mazumtirdzniecību specializētajos veikalos, kas klasificējas ar NACE
kodu <b>47.1</b>, sniedzot padziļinātu vakanču analīzi šajā apakšnozarē.
"""
story.append(Paragraph(market_overview_p3, intro_body_style))

# Add the final concluding paragraph for the section
market_overview_p4 = """
Analīzē izmantoti dati no populārākajiem darba meklēšanas portāliem, iekļaujot galvenos rādītājus
par vakanču <b>skaitu</b>, vidējo <b>atalgojumu</b> un to <b>ģeogrāfisko sadalījumu</b>. Šis pārskats kalpo kā pamats
konkurentu salīdzināšanai un palīdz pieņemt pamatotus, stratēģiskus lēmumus personāla vadībā.
"""
story.append(Paragraph(market_overview_p4, intro_body_style))


# Add a larger space before the next section
story.append(Spacer(1, 24))

print("General Market Overview section has been added to the story.")

General Market Overview section has been added to the story.


In [123]:
# ==============================================================================
# SECTION 3.1: VISPĀRĒJAIS MAZUMTIRDZNIECĪBAS TIRGUS [CORRECTED WIDOW CONTROL]
# ==============================================================================

# --- Step 1: Dynamically determine the General NACE Category for the selected company ---
try:
    general_nace_category = df_nace_codes[df_nace_codes['registration_number'] == selected_company_reg_num]['category'].iloc[0]
    print(f"Dynamically determined general NACE category: {general_nace_category}")
except (IndexError, KeyError) as e:
    print(f"Warning: Could not determine general NACE category for {selected_company_reg_num}. Falling back to '47'. Error: {e}")
    general_nace_category = '47'

# --- Step 2: Group the title with the first chart to prevent widows ---
# [THE FIX] Create a temporary list to hold the elements that must stay together.
title_and_first_chart = []

# Add the title to this temporary list
title_and_first_chart.append(Paragraph(f"3.1. Vispārējais mazumtirdzniecības tirgus (NACE kods {general_nace_category})", subsection_title_style))
title_and_first_chart.append(Spacer(1, 12))

# --- Chart 1: Total Vacancy Count (General Retail) ---
vacancy_count_data_gen = prepare_trend_data(
    base_jobs_df=df_darbnesis_job, monthly_assignments_df=df_month_assignments,
    nace_category=general_nace_category, 
    grouping_cols=['Month'], 
    aggregation_mode='count', y_col_name='Vakanču skaits'
)
if not vacancy_count_data_gen.empty:
    vacancy_count_data_gen['category'] = 'Vakanču skaits'
    block = ReportBlock(
        usable_width=USABLE_WIDTH, 
        block_title="Vakanču skaits (pēd. 3 mēn.)", 
        block_title_style=chart_title_style
    )
    elements = block.render_line_chart(
        chart_data=vacancy_count_data_gen, x_col='Mēnesis', y_col='Vakanču skaits', category_col='category',
        selected_company='Vakanču skaits', highlight_color='#05a1b4', color_palette=COLOR_PALETTE,
        x_axis_title='Mēnesis', y_axis_title='Kopējais vakanču skaits', show_legend=False, width_percentage=0.8,
        aspect_ratio=0.5 
    )
    # Add the first chart to the temporary list
    title_and_first_chart.extend(elements)

# Add the combined group to the main story, wrapped in KeepTogether
story.append(KeepTogether(title_and_first_chart))
story.append(Spacer(1, 18))


# --- Step 3: Add the remaining charts directly to the story ---

# --- Chart 2: Average Salary (General Retail) ---
avg_salary_data_gen = prepare_trend_data(
    base_jobs_df=df_darbnesis_job, monthly_assignments_df=df_month_assignments,
    nace_category=general_nace_category, 
    grouping_cols=['Month'], 
    aggregation_mode='mean', aggregation_col='AverageMonthlySalary', y_col_name='Vidējā mēnešalga (€)'
)
if not avg_salary_data_gen.empty:
    avg_salary_data_gen['category'] = 'Vidējā alga'
    block = ReportBlock(
        usable_width=USABLE_WIDTH, 
        block_title="Vidējā alga (pēd. 3 mēn.)", 
        block_title_style=chart_title_style
    )
    elements = block.render_line_chart(
        chart_data=avg_salary_data_gen, x_col='Mēnesis', y_col='Vidējā mēnešalga (€)', category_col='category',
        selected_company='Vidējā alga', highlight_color='#05a1b4', color_palette=COLOR_PALETTE,
        x_axis_title='Mēnesis', y_axis_title='Vidējā mēnešalga (€)', show_legend=False, width_percentage=0.8,
        aspect_ratio=0.5
    )
    story.extend(elements)
    story.append(Spacer(1, 18))

# --- Chart 3: Vacancies by Location (General Retail) ---
location_data_gen = prepare_trend_data(
    base_jobs_df=df_darbnesis_job, monthly_assignments_df=df_month_assignments,
    nace_category=general_nace_category, 
    grouping_cols=['Month', 'IrRīga'], 
    aggregation_mode='count', y_col_name='Vakanču skaits', category_col_name='Atrašanās vieta'
)
if not location_data_gen.empty:
    block = ReportBlock(
        usable_width=USABLE_WIDTH,
        block_title="Vakanču sadalījums: Rīga vs. reģioni (pēd. 3 mēn.)",
        block_title_style=chart_title_style
    )
    elements = block.render_line_chart(
        chart_data=location_data_gen, x_col='Mēnesis', y_col='Vakanču skaits', category_col='Atrašanās vieta',
        selected_company=None, highlight_color=HIGHLIGHT_COLOR, color_palette=COLOR_PALETTE,
        x_axis_title='Mēnesis', y_axis_title='Kopējais vakanču skaits', show_legend=True, width_percentage=0.8,
        aspect_ratio=0.5
    )
    story.extend(elements)

print(f"\nSection 3.1 (NACE: {general_nace_category}) has been built with widow control and added to the story.")

Dynamically determined general NACE category: 47
--- Preparing data for aggregation: mode='count', group_by=['Month'] ---
Filtering on NACE category: 47
Data preparation complete.

--- Preparing data for aggregation: mode='mean', group_by=['Month'] ---
Filtering on NACE category: 47
Data preparation complete.

--- Preparing data for aggregation: mode='count', group_by=['Month', 'IrRīga'] ---
Filtering on NACE category: 47
Data preparation complete.


Section 3.1 (NACE: 47) has been built with widow control and added to the story.


In [124]:
# ==============================================================================
# SECTION 3.2: APĢĒRBU MAZUMTIRDZNIECĪBA [FINAL LAYOUT FIX]
# ==============================================================================

# --- Step 1: Add all components directly to the main story ---
# [THE FIX] We are no longer using a temporary list or a single large KeepTogether wrapper.

# Add the title and introductory paragraph
story.append(Paragraph("3.2. Apģērbu mazumtirdzniecība specializētajos veikalos vakanču analīze (NACE kods 47.1)", subsection_title_style))
story.append(Paragraph(
    """
    Šajā sadaļā apskatītas darba tirgus tendences uzņēmumos, kas darbojas apģērbu mazumtirdzniecībā
    specializētajos veikalos. Dati aptver laika periodu no 2025. gada aprīļa līdz jūnijam un atspoguļo
    vakanču skaita, lokācijas un vidējās algas dinamiku. Svarīgi uzsvērt, ka šāda veida dati sniedz ieskatu
    par nozares aktivitāti un potenciālajām izmaiņām darbaspēka pieprasījumā.
    """,
    intro_body_style
))
story.append(Spacer(1, 12))

# --- Step 2: Get the specific NACE codes to use for filtering ---
try:
    specialized_nace_codes = df_nace_codes[df_nace_codes['registration_number'] == selected_company_reg_num]['nace_codes'].unique().tolist()
except Exception as e:
    specialized_nace_codes = []

# --- Step 3: Generate the charts and their full descriptions ---
if specialized_nace_codes:
    # --- Chart 1: Total Vacancy Count (Specialized) ---
    vacancy_count_data = prepare_trend_data(
        base_jobs_df=df_darbnesis_job, monthly_assignments_df=df_month_assignments,
        specific_nace_codes=specialized_nace_codes, grouping_cols=['Month'], 
        aggregation_mode='count', y_col_name='Vakanču skaits'
    )
    if not vacancy_count_data.empty:
        vacancy_count_data['category'] = 'Vakanču skaits'
        block = ReportBlock(
            usable_width=USABLE_WIDTH, 
            block_title="Vakanču skaits (pēd. 3 mēn.)", 
            block_title_style=chart_title_style
        )
        elements = block.render_line_chart(
            chart_data=vacancy_count_data, x_col='Mēnesis', y_col='Vakanču skaits', category_col='category',
            selected_company='Vakanču skaits', highlight_color='#05a1b4', color_palette=COLOR_PALETTE,
            x_axis_title='Mēnesis', y_axis_title='Kopējais vakanču skaits', show_legend=False, width_percentage=0.8,
            aspect_ratio=0.5 # Using the smaller aspect ratio
        )
        story.extend(elements)
        
        story.append(Paragraph(
            """
            Šis grafiks ilustrē kopējo vakanču skaitu nozarē laika gaitā. Dati <b>uzrāda lēnu, bet pakāpenisku
            samazinājumu</b> no aprīļa līdz jūnijam. Šī tendence var liecināt par vairākām iespējamām norisēm —
            sezonālās darbaspēka vajadzības var būt jau apmierinātas, vai arī uzņēmumi kļūst selektīvāki un
            samazina aktīvo sludinājumu skaitu vasaras sezonā. Samazinājums var būt saistīts arī ar <b>darba tirgus
            piesātinājumu</b> vai <b>lielāku darbinieku noturību</b> pēc pavasara pieņemšanas cikliem.
            """,
            intro_body_style
        ))
        story.append(Spacer(1, 18))

    # --- Chart 2: Average Salary (Specialized) ---
    avg_salary_data = prepare_trend_data(
        base_jobs_df=df_darbnesis_job, monthly_assignments_df=df_month_assignments,
        specific_nace_codes=specialized_nace_codes, grouping_cols=['Month'], 
        aggregation_mode='mean', aggregation_col='AverageMonthlySalary', y_col_name='Vidējā mēnešalga (€)'
    )
    if not avg_salary_data.empty:
        avg_salary_data['category'] = 'Vidējā alga'
        block = ReportBlock(
            usable_width=USABLE_WIDTH, 
            block_title="Vidējā alga (pēd. 3 mēn.)", 
            block_title_style=chart_title_style
        )
        elements = block.render_line_chart(
            chart_data=avg_salary_data, x_col='Mēnesis', y_col='Vidējā mēnešalga (€)', category_col='category',
            selected_company='Vidējā alga', highlight_color='#05a1b4', color_palette=COLOR_PALETTE,
            x_axis_title='Mēnesis', y_axis_title='Vidējā mēnešalga (€)', show_legend=False, width_percentage=0.8,
            aspect_ratio=0.5 # Using the smaller aspect ratio
        )
        story.extend(elements)

        story.append(Paragraph(
            """
            Šajā grafikā redzams, ka vidējā mēnešalga analizētajā nozarē (NACE 47.71) ir <b>stabila,</b> bez būtiskām
            svārstībām — <b>aptuveni 1 000 EUR mēnesī.</b> Šī stabilitāte norāda uz prognozējamu un strukturētu
            atalgojuma politiku šajā nozarē. Tomēr, svarīgi ņemt vērā iespējamos <b>slodzes ietekmi uz norādīto
            summu:</b> darba devēji ne vienmēr precizē, vai minētais atalgojums attiecas uz pilnu slodzi, kas var
            radīt neobjektīvu priekšstatu par reālo atalgojumu konkrētajā pozīcijā. Tāpēc šos datus jāinterpretē
            piesardzīgi un vēlams kontekstā ar sludinājuma saturu.
            """,
            intro_body_style
        ))
        story.append(Spacer(1, 18))

    # --- Chart 3: Vacancies by Location (Specialized) ---
    location_data = prepare_trend_data(
        base_jobs_df=df_darbnesis_job, monthly_assignments_df=df_month_assignments,
        specific_nace_codes=specialized_nace_codes, grouping_cols=['Month', 'IrRīga'], 
        aggregation_mode='count', y_col_name='Vakanču skaits', category_col_name='Atrašanās vieta'
    )
    if not location_data.empty:
        block = ReportBlock(
            usable_width=USABLE_WIDTH,
            block_title="Vakanču sadalījums: Rīga vs. reģioni (pēd. 3 mēn.)",
            block_title_style=chart_title_style
        )
        elements = block.render_line_chart(
            chart_data=location_data, x_col='Mēnesis', y_col='Vakanču skaits', category_col='Atrašanās vieta',
            selected_company=None, highlight_color=HIGHLIGHT_COLOR, color_palette=COLOR_PALETTE,
            x_axis_title='Mēnesis', y_axis_title='Kopējais vakanču skaits', show_legend=True, width_percentage=0.8,
            aspect_ratio=0.5 # Using the smaller aspect ratio
        )
        story.extend(elements)

        story.append(Paragraph(
            """
            Šis grafiks attēlo vakanču sadalījumu pēc lokācijas — <b>Rīgā un ārpus Rīgas</b> — laika griezumā pa
            mēnešiem. Dati skaidri norāda, ka <b>lielākā daļa vakanču šajā nozarē koncentrējas Rīgā,</b> kur darba
            iespēju skaits ir būtiski augstāks nekā reģionos. Gan Rīgā, gan ārpus tās vakanču skaits maija un jūnija
            laikā nedaudz samazinās, kas var būt saistāms ar sezonālajām izmaiņām, piemēram, pavasara beigu
            un vasaras sākuma periodiem, kad darbinieku plūsma tradicionāli kļūst stabilāka, un uzņēmumi
            mazāk aktīvi veic personāla atlasi. Tas var būt arī saistīts ar augstāku kandidātu noturību pēc sezonas
            pieņemšanām vai pieprasījuma kritumu vasaras sākumā.
            """,
            intro_body_style
        ))

print("\nSection 3.2 has been built with flexible layout and added to the story.")

--- Preparing data for aggregation: mode='count', group_by=['Month'] ---
Filtering on specific NACE codes: ['47.71']
Data preparation complete.

--- Preparing data for aggregation: mode='mean', group_by=['Month'] ---
Filtering on specific NACE codes: ['47.71']
Data preparation complete.

--- Preparing data for aggregation: mode='count', group_by=['Month', 'IrRīga'] ---
Filtering on specific NACE codes: ['47.71']
Data preparation complete.


Section 3.2 has been built with flexible layout and added to the story.


In [125]:
# ==============================================================================
# SECTION 4: KONKURENTU ANALĪZE
# ==============================================================================

# --- Add the title for the new section ---
story.append(Spacer(1, 24))
section_title_4 = "4. Konkurentu analīze"
story.append(Paragraph(section_title_4, intro_title_style))
story.append(Spacer(1, 6))

# --- Add the introductory paragraphs ---
text_block_4_1 = """
Turpmākajās sadaļās apkopota <b>Apranga, LPP Latvia, H&M Hennes & Mauritz SIA, Danija un citu
konkurentu</b> – piedāvāto vakanču skaita, algas un populārāko priekšrocību analīze. Šie vizuālie dati
ļauj objektīvi izvērtēt gan darba devēju priekšrocību piedāvājumu, gan kandidātu iespējas dažādos
Latvijas reģionos.
"""
story.append(Paragraph(text_block_4_1, intro_body_style))

text_block_4_2 = """
Šajā sekcijā detalizēti salīdzinātas Apranga galveno konkurentu personāla piesaistes stratēģijas,
vakances un piedāvātie nosacījumi. Analīze balstās uz pēdējo 3 mēnešu laikā publicētajām vakancēm,
akcentējot gan atalgojuma līmeņus, gan biežāk piedāvātos bonusus, darba grafika elastību un citas
priekšrocības, ko darba devēji izmanto kandidātu piesaistei.
"""
story.append(Paragraph(text_block_4_2, intro_body_style))

text_block_4_3 = """
Papildus tiek izvērtēts, kādi amata veidi un nosaukumi bijuši aktuāli pie katra konkurenta, kā arī,
kādas atšķirības vērojamas reģionālā un amata specifikā. Sadaļā iekļauti arī vizuāli salīdzinājumi –
atalgojuma, bonusu un priekšrocību tabulas, kas ļauj ātri novērtēt Apranga konkurētspēju darba tirgū
attiecīgajās pozīcijās.
"""
story.append(Paragraph(text_block_4_3, intro_body_style))

text_block_4_4 = """
Šī informācija sniedz vispusīgu ieskatu par galveno konkurentu piedāvājumiem un palīdz identificēt
jomas, kurās Apranga var uzlabot vai izcelt savu darba devēja vērtību potenciālo kandidātu acīs.
"""
story.append(Paragraph(text_block_4_4, intro_body_style))

# Add a larger spacer before the content that will follow
story.append(Spacer(1, 12))

print("Introduction for Section 4 has been added to the story.")

Introduction for Section 4 has been added to the story.


In [126]:
# ==============================================================================
# SECTION 4.1: VAKANČU SKAITS UN SADALĪJUMS [FINAL LAYOUT FIX]
# ==============================================================================

# --- Step 1: Prepare Base Data (used by all charts in this section) ---
competitor_reg_numbers = df_kompanijas_konkurenti['Competitor registration number'].unique().tolist()
all_relevant_reg_numbers = competitor_reg_numbers + [selected_company_reg_num]
competitor_jobs_df = df_darbnesis_job[df_darbnesis_job['registration_number'].isin(all_relevant_reg_numbers)].copy()
today = pd.to_datetime('today')
end_date = pd.Timestamp(today.year, today.month, 1)
start_date = end_date - pd.DateOffset(months=3)
recent_months_df = df_month_assignments[(pd.to_datetime(df_month_assignments['MonthAssignment']) >= start_date) & (pd.to_datetime(df_month_assignments['MonthAssignment']) < end_date)]
active_competitor_jobs = pd.merge(competitor_jobs_df, recent_months_df[['id']], on='id', how='inner')
apranga_name = df_alias_list[df_alias_list['registration_number'] == selected_company_reg_num]['official_name'].iloc[0]

# --- Add all components directly to the main story for flexible layout ---

# Add the main section title
story.append(Paragraph("4.1. Vakanču skaits un sadalījums pēc pozīcijām", subsection_title_style))
story.append(Spacer(1, 12))

# Component 1: Bar Chart
job_counts_by_reg = active_competitor_jobs.groupby('registration_number')['id'].nunique()
job_counts_df = pd.merge(job_counts_by_reg.reset_index(), df_distinct_orgs, on='registration_number', how='left')
chart_df = job_counts_df.set_index('official_name')[['id']].rename(columns={'id': 'Vakanču skaits'}).sort_values('Vakanču skaits', ascending=False)
if not chart_df.empty:
    max_count = chart_df['Vakanču skaits'].iloc[0]
    threshold = max_count * 0.03
    significant_chart_df = chart_df[chart_df['Vakanču skaits'] >= threshold]
    chart_df_prepared = prepare_data_with_highlight(df=significant_chart_df, full_color_palette=COLOR_PALETTE, selected_company=apranga_name, highlight_color=HIGHLIGHT_COLOR)
    chart_df_final = wrap_df_index_labels(df=chart_df_prepared, usable_chart_width=int(USABLE_WIDTH), approx_char_pixel_width=4, padding_per_bar=8, min_wrap_limit=15)
    
    bar_chart_block = ReportBlock(
        usable_width=USABLE_WIDTH,
        block_title="Vakanču skaits pēc uzņēmumiem",
        block_title_style=chart_title_style,
    )
    story.extend(bar_chart_block.render_bar_chart(chart_data=chart_df_final))

# Component 2: Descriptive Text
story.append(Paragraph(
    """
    (Grafiks nr. 5) Analizējot vakanto darba vietu skaitu dažādās mazumtirdzniecības nozares
    mātes kompānijās Latvijā, redzams, ka uzņēmums Apranga ar 24 vakancēm ierindojas
    konkurētspējīgā pozīcijā, atrodoties tuvu piektajai vietai starp vadošajiem darba devējiem
    šajā segmentā. Visvairāk vakanču šobrīd piedāvā LPP Latvia (57), kam seko Sportland (33),
    Poldma Kaubanduse Aktsiaselts (27) un New Yorker (25). Šie dati norāda uz augstu aktivitāti
    darbaspēka piesaistē šajos uzņēmumos, taču Apranga saglabā stabilu pozīciju ar līdzvērtīgu
    vakanču skaitu. Pārējie tirgus dalībnieki, tostarp Stockmann, Humana, Lindex un H&M;,
    uzrāda nedaudz zemāku rādītāju, savukārt tādi uzņēmumi kā CCC Shoes, Danija, WEEKEND,
    Eiropas apavi un SIA Peek & Cloppenburg pašlaik piedāvā ievērojami mazāku vakanto vietu
    skaitu. Diagrammā attēlotie dati sniedz skaidru ieskatu par kopējo situāciju
    mazumtirdzniecības nodarbinātības tirgū un ļauj Apranga novērtēt savu pozīciju
    konkurences kontekstā, kā arī identificēt potenciālās iespējas darbaspēka piesaistes
    stratēģijas uzlabošanai.
    """,
    intro_body_style
))
story.append(Spacer(1, 18))

# Component 3: Title for Matrix and Ribbon Chart
story.append(Paragraph("Vakanču skaits pa mēnešiem pēc uzņēmumiem", chart_title_style))
story.append(Spacer(1, 6))

# Component 4: Matrix Visual
active_competitor_jobs_monthly = pd.merge(
    active_competitor_jobs[['id', 'registration_number']],
    recent_months_df[['id', 'Month']],
    on='id', how='inner'
).drop_duplicates()
active_competitor_jobs_monthly = pd.merge(active_competitor_jobs_monthly, df_distinct_orgs, on='registration_number', how='left')
matrix_block = ReportBlock(usable_width=USABLE_WIDTH)
story.extend(matrix_block.render_matrix_table(
    chart_data=active_competitor_jobs_monthly,
    index_col='official_name',
    columns_col='Month',
    values_col='id'
))

# Component 5: Ribbon Visual
top_companies = chart_df.head(8).index.tolist()
ribbon_df_filtered = active_competitor_jobs_monthly[active_competitor_jobs_monthly['official_name'].isin(top_companies)]
ribbon_pivot = ribbon_df_filtered.pivot_table(index='official_name', columns='Month', values='id', aggfunc='nunique').fillna(0).astype(int)
month_order = ['April', 'May', 'June']
present_months = [m for m in month_order if m in ribbon_pivot.columns]
ribbon_pivot = ribbon_pivot[present_months]
ribbon_pivot['Total'] = ribbon_pivot.sum(axis=1)
ribbon_pivot = ribbon_pivot.sort_values(by='Total', ascending=False).drop(columns='Total')
other_colors = [color for color in COLOR_PALETTE if color != HIGHLIGHT_COLOR]
random.shuffle(other_colors)
color_map = {}
color_idx = 0
for company in ribbon_pivot.index:
    if company == apranga_name:
        color_map[company] = HIGHLIGHT_COLOR
    else:
        color_map[company] = other_colors[color_idx % len(other_colors)]
        color_idx += 1
ribbon_block = ReportBlock(usable_width=USABLE_WIDTH)
story.extend(ribbon_block.render_ribbon_chart(chart_data=ribbon_pivot, colors=color_map))

# Component 6: Ribbon Chart Explanatory Text
story.append(Paragraph("(Grafiks nr. 6)", intro_body_style))
story.append(Spacer(1, 18))
story.append(Paragraph("Kā lasīt grafiku:", sub_heading_style))
bullets = ["Katra krāsainā josla (lente) attēlo vakanču skaitu konkrētam uzņēmumam katrā mēnesī.", "Grafiks ļauj ērti izsekot, kā mainās kopējais vakanču apjoms un kā katra uzņēmuma daļa tajā svārstās laika gaitā."]
for point in bullets:
    story.append(Paragraph(f"• {point}", bullet_style))
story.append(Spacer(1, 12))
story.append(Paragraph("Šis grafiks ilustrē vakanču skaita izmaiņas dažādās uzņēmumu grupās trīs mēnešu periodā - 2025. gada aprīlī, maijā un jūnijā. LPP Latvia (rozā krāsa) ir ar lielāko vakanču skaitu katrā mēnesī, taču šis skaits samazinās no 41 aprīlī līdz 37 maijā un 25 jūnijā, kas norāda uz sezonālu svārstību vai samazinātu pieprasījumu vasarā. Sportland (zils), Poldma (sarkans) un Apranga (tumši zils) saglabā salīdzinoši stabilu vakanču skaitu visā periodā, kas liecina par līdzsvarotu pieprasījumu šajos uzņēmumos. Citu uzņēmumu, piemēram, Stockmann, Humana, Lindex un New Yorker, vakanču skaits ir zemāks, taču arī šeit vērojama līdzīga tendence - maza svārstība vai neliels kritums vasaras sākumā.", intro_body_style))
story.append(Spacer(1, 18))

# --- [THE FIX] Component 7: Side-by-Side Location Comparison Charts ---
def create_simple_bar_image(chart_df, width, height):
    """A simplified local function to create a bar chart image, avoiding ReportBlock."""
    max_val = chart_df[chart_df.columns[0]].max()
    fig = go.Figure(go.Bar(
        x=chart_df.index, y=chart_df[chart_df.columns[0]],
        marker=dict(color=chart_df['barcolor'], cornerradius=10),
        texttemplate='<b>%{y}</b>', textposition='outside',
        outsidetextfont=dict(family='Ubuntu', size=24, color='#404040'),
        cliponaxis=False
    ))
    fig.update_layout(
        font=dict(family="Ubuntu", color="#404040"), plot_bgcolor='white', paper_bgcolor='white',
        showlegend=False, margin=dict(l=20, r=20, t=5, b=20),
        yaxis=dict(showgrid=False, showticklabels=False, range=[0, max_val * 1.4])
    )
    img_bytes = fig.to_image(format="png", width=width*3, height=height*3, scale=3)
    return Image(BytesIO(img_bytes), width=width, height=height)

# Prepare data for both charts
df_apranga = active_competitor_jobs[active_competitor_jobs['registration_number'] == selected_company_reg_num]
df_competitors = active_competitor_jobs[active_competitor_jobs['registration_number'] != selected_company_reg_num]
apranga_counts = df_apranga.groupby('IrRīga')['id'].nunique()
competitor_counts = df_competitors.groupby('IrRīga')['id'].nunique()
riga_data = pd.DataFrame({'Vakanču skaits': [apranga_counts.get('Rīga', 0), competitor_counts.get('Rīga', 0)]}, index=[apranga_name, 'Konkurenti'])
riga_data_prepared = prepare_data_with_highlight(riga_data, [COLOR_PALETTE[0]], apranga_name, HIGHLIGHT_COLOR)
arpus_rigas_data = pd.DataFrame({'Vakanču skaits': [apranga_counts.get('Ārpus Rīgas', 0), competitor_counts.get('Ārpus Rīgas', 0)]}, index=[apranga_name, 'Konkurenti'])
arpus_rigas_data_prepared = prepare_data_with_highlight(arpus_rigas_data, [COLOR_PALETTE[0]], apranga_name, HIGHLIGHT_COLOR)

# Create the chart images and titles
chart_width = (USABLE_WIDTH / 2) * 0.9
chart_height = chart_width * 0.8
image_riga = create_simple_bar_image(riga_data_prepared, chart_width, chart_height)
image_arpus = create_simple_bar_image(arpus_rigas_data_prepared, chart_width, chart_height)
title_riga = Paragraph("Vakances Rīgā", chart_title_style)
title_arpus = Paragraph("Vakances ārpus Rīgas", chart_title_style)

# A table cell can safely contain a list of simple flowables.
cell_content_riga = [title_riga, Spacer(1, 6), image_riga]
cell_content_arpus = [title_arpus, Spacer(1, 6), image_arpus]

# Place the lists of flowables into the table
location_chart_table = Table([[cell_content_riga, cell_content_arpus]], colWidths=[USABLE_WIDTH/2, USABLE_WIDTH/2])
location_chart_table.setStyle(TableStyle([('VALIGN', (0,0), (-1,-1), 'TOP')]))
story.append(location_chart_table)

# Component 8: Location Chart Explanatory Text
story.append(Paragraph("(Grafiks nr. 7)", intro_body_style))
story.append(Spacer(1, 12))
story.append(Paragraph("Analizējot vakanču ģeogrāfisko sadalījumu, redzams, ka <b>Apranga</b> lielāko daļu darba iespēju koncentrē <b>Rīgā, kur šobrīd pieejamas 23 vakances,</b> savukārt <b>ārpus Rīgas</b> norādītas tikai <b>2 vakances.</b> Salīdzinājumam — <b>konkurējošie uzņēmumi</b> Rīgā piedāvā <b>192</b> vakances, bet ārpus tās vēl <b>50,</b> kas norāda uz plašāku darbības pārklājumu arī reģionos.", intro_body_style))
story.append(Paragraph("Šī atšķirība tomēr jāskata kontekstā ar uzņēmuma darbības struktūru — <b>Aprangai Latvijā šobrīd ir tikai viens veikals ārpus Rīgas, kas atrodas Jūrmalā.</b> Līdz ar to zemo vakanču skaitu ārpus galvaspilsētas loģiski nosaka arī ierobežotais veikalu skaits šajās teritorijās.", intro_body_style))
story.append(Paragraph("Dati kopumā atspoguļo galvaspilsētas orientētu darbaspēka piesaistes modeli, kas šobrīd ir pilnībā atbilstošs esošajai tirdzniecības tīkla struktūrai, taču vienlaikus var <b>kalpot</b> kā bāze turpmākai <b>diskusijai</b> par potenciālu <b>izaugsmi</b> arī reģionālajos centros.", intro_body_style))

print("\nSection 4.1, including corrected location analysis, has been built and added to the story.")

Dynamically calculated character limit for wrapping: 15

Section 4.1, including corrected location analysis, has been built and added to the story.


In [127]:
# ==============================================================================
# SECTION 4.1 (CONTINUED): VAKANČU SKAITS PĒC POZĪCIJAS (ISCO KODI) [CORRECTED]
# ==============================================================================

# --- Add all components directly to the story for flexible layout ---

# Add the title for this new analysis
story.append(Paragraph("Vakanču skaits pēc pozīcijas", chart_title_style))
story.append(Spacer(1, 6))

# --- Step 1: Prepare data for the Pie Chart ---
# Use 'active_competitor_jobs' and get the value counts of the ISCO codes
isco_counts = active_competitor_jobs.dropna(subset=['Isco kods'])['Isco kods'].astype(int).astype(str).value_counts()

# --- Step 2: Render the Pie Chart (with correct colors) ---
pie_chart_block = ReportBlock(
    usable_width=USABLE_WIDTH,
)
# [THE FIX] Passing the COLOR_PALETTE from the config to the render method.
story.extend(pie_chart_block.render_pie_chart(
    chart_data=isco_counts,
    colors=COLOR_PALETTE
))

# --- Step 3: Add the explanatory text ---
story.append(Paragraph("(Grafiks nr. 8)", intro_body_style))
story.append(Spacer(1, 12))

text_1 = """
Sektoru diagrammā redzams, ka pārliecinoši lielākā daļa vakanču <b>(61,94%) atbilst ISCO-08
kodam 5223,</b> kas klasificē mazumtirdzniecības pārdevējus. Tas liecina par augstu pieprasījumu
pēc pārdošanas darbiniekiem un <b>uzsver</b> šo pozīciju kā dominējošo lomu
mazumtirdzniecības sektorā.
"""
story.append(Paragraph(text_1, intro_body_style))

text_2 = """
Otrā lielākā daļa – <b>13,77%</b> – ir saistīta ar kodu
<b>1420,</b> kas attiecas uz mazumtirdzniecības veikalu vadītājiem. Tam seko vairākas citas
pozīcijas ar salīdzinoši <b>nelielu</b> īpatsvaru,
piemēram, <b>5221 (3,64%), 5222 (3,64%), 1221
(2,02%), 1222 (2,02%)</b> un citas. Šie dati liecina
par daudzveidīgu, bet koncentrētu
pieprasījumu – dominējošais vairākums
vakanču ir operatīva līmeņa <b>amatos</b> ar tiešu kontaktu ar klientu.
"""
story.append(Paragraph(text_2, intro_body_style))

text_3 = """
Jāatzīmē, ka pārējās pozīcijas, piemēram, <b>administratīvie darbinieki, speciālisti un vadības līmeņa
darbinieki,</b> katra veido mazāk nekā 5% no kopējā vakanču apjoma, kas norāda uz ierobežotu
vajadzību pēc šiem amatiem vai to retāku norādīšanu darba sludinājumos.
"""
story.append(Paragraph(text_3, intro_body_style))

text_4 = """
Šāds sadalījums sniedz skaidru <b>ieskatu</b> tajā, ka mazumtirdzniecības nozare šobrīd koncentrējas
galvenokārt uz darbinieku <b>piesaisti</b> pirmās līnijas pārdošanas pozīcijām, savukārt <b>citu</b> lomu
pieejamība ir daudz ierobežotāka.
"""
story.append(Paragraph(text_4, intro_body_style))

print("\nISCO code analysis (pie chart) has been corrected and added to Section 4.1.")


ISCO code analysis (pie chart) has been corrected and added to Section 4.1.


## we print the story

In [128]:
# 3. Generate the document using SimpleDocTemplate
# It works perfectly now that our images are sized correctly for its frame.
doc = SimpleDocTemplate(
    OUTPUT_PDF_PATH,
    pagesize=letter,
    leftMargin=MARGIN,
    rightMargin=MARGIN,
    topMargin=MARGIN,
    bottomMargin=MARGIN
)
doc.build(story)

print(f"Report '{OUTPUT_PDF_PATH}' generated successfully.")

Report '../Report builder/Reports/report_from_notebook.pdf' generated successfully.
